In [1]:
from weiner_variation.sim.data_structures import DrawInput
from weiner_variation.sim.process import PASS_SEQUENCE, DIAMETER, TEMPERATURE, DIAMETER_STD, TEMPERATURE_STD, IN_PROFILE
from weiner_variation.sim.config import SAMPLE_COUNT, FIELDS, SEED
from weiner_variation.config import DATA_DIR

import pandas as pd
import numpy as np
from scipy.stats import norm
from copy import deepcopy
import tqdm
from multiprocessing import Pool
import os

/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (depth - l23(_alpha) * np.sin(_alpha) - fh(_alpha)) / (1 - np.cos(_alpha))
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:83: RuntimeWarning: invalid value encountered in scalar multiply
  width / 2 - _r2 * np.sin(_alpha) - l23(_alpha) * np.cos(_alpha)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:84: RuntimeWarning: invalid value encountered in scalar multiply
  - fw(_alpha) - (_r2 + r4) * np.sin(_alpha4)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (de

In [2]:
import pyroll.basic as pr

In [3]:
OUTPUT_FILENAME = "sim_nominal_results.csv"

In [4]:
sequence = deepcopy(PASS_SEQUENCE)
sequence.solve(IN_PROFILE)       

result = { ("draw", "diameter"): DIAMETER, ("draw", "temperature"): TEMPERATURE } | (
    FIELDS.items()
    |> starmap$((key, extractor) -> (
            sequence.units 
            |> filter$(u -> isinstance(u, pr.RollPass)) 
            |> map$(u -> ((key, u.label), extractor(u))) 
            |> filter$(t -> t[1] is not None)
        )
    )
    |> flatten
    |> dict            
)

In [30]:
df = result |> pd.Series |> .T |> .infer_objects()
df

draw        diameter            0.050000
            temperature      1423.150000
roll_force  R1             391317.673127
            R2             227464.097445
            R3             423776.866442
                               ...      
strain      R10                 0.459861
            F1                  0.342711
            F2                  0.346667
            F3                  0.447922
            F4                  0.416546
Length: 100, dtype: float64

In [32]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)